In [1]:
import numpy as np
import matplotlib.pyplot as plt

import data
import data.getSyntheticData as SyntheticData

from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.svm import OneClassSVM
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import precision_score, recall_score, roc_curve

from sklearn.kernel_approximation import Nystroem

from models.OCSMM import OCSMM
from utils.gridSearch import *
from utils.Nystroem_SMM import Nystroem_SMM

import pickle
import itertools


In [2]:
def compute_kernel(X, sigma):
    K = np.zeros((len(X), len(X)))
    rbf = RBF(length_scale=sigma)
    for (i, j) in itertools.combinations(range(len(X)), 2):
            kernel = rbf(X[i], X[j])
            n = X[i].shape[0]
            K[i, j] = np.sum(np.sum(kernel, axis=0), axis=0) / (n ** 2)
            K[j, i] = K[i, j]
    return K

def compute_cross_kernel(X, Y, sigma):
    K = np.zeros((len(X), len(Y)))
    rbf = RBF(length_scale=sigma)
    for (i, j) in itertools.product(range(len(X)), range(len(Y))):
            kernel = rbf(X[i], Y[j])
            n_i = X[i].shape[0]
            n_j = Y[j].shape[0]
            K[i, j] = np.sum(np.sum(kernel, axis=0), axis=0) / (n_i * n_j)
    return K

In [3]:
# Experiment plots
def plot_synthetic_data(Stest, scores):
    scores = np.squeeze(scores)
    score_plot = [np.array([sc]*Stest[idx].shape[0]) for idx, sc in enumerate(scores)]
    mini, maxi = -2.5, 2.5
    norm = plt.Normalize(mini, maxi)
    
    fig, axes = plt.subplots(2, 6, figsize=(15, 4.5), sharey=True)
    idx=0
    for ax in axes.flat:
        ax.scatter(Stest[idx][:,0], Stest[idx][:,1], 
                         c=score_plot[idx], cmap='viridis', norm=norm)
        idx+=1
        
    for a, color in zip([axes[1][9%6], axes[1][10%6], axes[1][11%6]], ['green', 'red', 'purple']):
        plt.setp(a.spines.values(), color=color, ls=':')
        plt.setp([a.get_xticklines(), a.get_yticklines()], color=color, ls=':')
    
    fig.tight_layout()
    fig.colorbar(plt.gca().get_children()[0], ax=axes.ravel().tolist()).set_label('Anomaly Score')
#     plt.savefig('synth_data.png')
    plt.show()

def plot_roc_curve(scores, Ytest):
    # Receives trained clasiffier and plots its roc curve
    # Against Test Set.
    p_values = empirical_p_values(scores)
    auc, best_cut, roc_curve = roc_auc(Ytest, p_values)
    plt.plot(roc_curve[:,0], roc_curve[:,1])
    plt.savefig('PR.png')
    plt.show()
    

In [ ]:
# Generate Synthetic Data
Strain = SyntheticData.genSyntheticTrainData(n_sample=100, n_sets=50)

Stest = SyntheticData.genSyntheticTrainData(n_sample=100, n_sets=9)
X_anomaly1 = np.random.multivariate_normal(mean = [0.0, 0.0], cov = np.identity(2), size=100)
# X_anomaly2 = np.random.multivariate_normal(mean = [0.0, 0.0], cov = np.identity(2), size=100)
# X_anomaly3 = np.random.multivariate_normal(mean = [0.0, 0.0], cov = np.identity(2), size=100)
X_anomaly2 = SyntheticData.genSyntheticSet(100, pmix=[0.33, 0.33, 0.33], sigma_c = 0.2)
X_anomaly3 = SyntheticData.genSyntheticSet(100, pmix=[0.33, 0.33, 0.33], sigma_c = 0.2)
Stest.extend([X_anomaly1, X_anomaly2, X_anomaly3])

Ytest = [1] * 9 + [-1] * 3

In [4]:
# Generate Synthetic Data 2
Strain = SyntheticData.genSyntheticTrainData_1(n_sample=100, n_sets=47)

Stest = SyntheticData.genSyntheticTrainData_1(n_sample=100, n_sets=9)
X_anomaly1 = SyntheticData.genSyntheticSet_1(100, pmix=[0.61, 0.1, 0.06, 0.23], seed=1)
X_anomaly2 = SyntheticData.genSyntheticSet_1(100, pmix=[0.61, 0.1, 0.06, 0.23], seed=2)
X_anomaly3 = SyntheticData.genSyntheticSet_1(100, pmix=[0.61, 0.1, 0.06, 0.23], seed=3)
Stest.extend([X_anomaly1, X_anomaly2, X_anomaly3])
Strain.extend([X_anomaly1, X_anomaly2, X_anomaly3])

Ytest = [1] * 9 + [-1] * 3

In [ ]:
# Strain = pickle.load(open('HiggsStrain.pkl', 'rb'))
# Stest = pickle.load(open('HiggsStest.pkl', 'rb'))
# Ytest = pickle.load(open('HiggsYtest.pkl', 'rb'))
# Ytest = [1 - 2 * y for y in Ytest]
# print(len(Strain))
# print(len(Stest))

In [ ]:
precision = np.zeros((10, 10))
recall = np.zeros((10, 10))
for i, sigma in enumerate([0.1 * i for i in range(1, 2)]):
    K = compute_kernel(Strain, sigma)
    Ktest = compute_cross_kernel(Stest, Strain, sigma)
    for j, nu in enumerate([0.1 * i for i in range(1, 2)]):
        ocsmm = OneClassSVM(kernel='precomputed', nu=nu)
        Ypred_train = ocsmm.fit_predict(K)
        scores_train = ocsmm.decision_function(K)
        Ypred = ocsmm.predict(Ktest)
        scores = ocsmm.decision_function(Ktest)
#         precision[i, j] = precision_score(Ytest, Ypred)
#         recall[i, j] = recall_score(Ytest, Ypred)
#         print('sigma: %f, nu: %f, precision: %f, recall: %f' % (sigma, nu, precision[i, j], recall[i, j]))
#     plt.figure()
#     plt.plot(recall[i, :], precision[i, :])
# for i in range(10):

print(scores)
fpr, tpr, th = roc_curve(Ytest, scores)
plt.plot(fpr, tpr)
plot_synthetic_data(Stest, -100 * scores)
plot_synthetic_data(Strain[38:50], -100 * scores_train[38:50])
fpr, tpr, th = roc_curve([1] * 47 + [-1] * 3, scores_train)
plt.plot(fpr, tpr)
print(scores_train)

In [ ]:
print(len(Strain))

In [ ]:
plt.figure()
plt.plot(recall[1, :-1], precision[1, :-1])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.savefig('PR.png')

In [ ]:
print(scores)
print(Ypred)
print(Ytest)

In [5]:
def mean_map_kernel(X, Y, sigma):
    X = X.reshape(-1, 2)
    Y = Y.reshape(-1, 2)
    rbf = RBF(length_scale=sigma)
    kernel = rbf(X, Y)
    n_i = X.shape[0]
    n_j = Y.shape[0]
    K = np.sum(np.sum(kernel, axis=0), axis=0) / (n_i * n_j)
    return K

n_components = 10
sigma = 0.1
nyst = Nystroem_SMM(kernel=compute_cross_kernel, kernel_params=sigma, n_components=n_components)
data_transformed = nyst.fit_transform(np.array(Strain))
ocsvm = OneClassSVM(kernel='linear', nu=0.1)
Ypred = ocsvm.fit_predict(data_transformed)

In [6]:
print(Ypred)

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1 -1  1 -1  1  1  1  1  1  1  1  1 -1
 -1  1]
